# Financial ML

In [4]:
import numpy as np
import pandas as pd
import sklearn
import datetime

In [5]:
try:
    from pydrive.auth import GoogleAuth
    from pydrive.drive import GoogleDrive
    from google.colab import auth
    from oauth2client.client import GoogleCredentials
    auth.authenticate_user()
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    drive = GoogleDrive(gauth)
    link = 'https://drive.google.com/open?id=1Xl9krWLvAUF0IMxFA0hqF2HwG5AejwTN'
    fluff, id = link.split('=')
    downloaded = drive.CreateFile({'id':id}) 
    downloaded.GetContentFile('hourAll.csv')  
    data = pd.read_csv('hourAll.csv')
except:
    print('cant colab')

cant colab


In [9]:
try:
    from google.colab import files
    uploaded = files.upload()
except:
    print('cant colab')

cant colab


## Data

In [10]:
class TAQ():
    """TAQ object generated from WRDS database TAQ trade csv file.
    Stores initial CSV as pd dataframe and renames columns"""

    def __init__(self, path='null', data=None):
        if path != 'null':
            self.taqPath = path
            self.rawData = pd.read_csv(path)
        elif data is not None:
            self.rawData = data

        self.data = self.preprocess()

    def make_timestamp(self, level = 'Min'):
        combined = self.rawData['DATE'].apply(str) + ' ' + self.rawData['TIME_M']
        timestamp = pd.to_datetime(combined)
        return timestamp.dt.floor(level)

    def preprocess(self, level = 'Min'):
        data = self.rawData[['SYM_ROOT', 'EX', 'SIZE', 'PRICE']]
        data = data.rename(columns={"SYM_ROOT": "ticker", "EX": 'exchange', "SIZE" : 'volume', "PRICE": 'price'})
        data['timestamp'] = self.make_timestamp(level)
        data['date'] = data['timestamp'].dt.date
        return data

In [11]:
try:
    import io
    taq = TAQ(io.BytesIO(uploaded['QQQ1Day.csv']))
except:
    taq = TAQ('QQQ1Day.csv')

In [12]:
rawData = taq.rawData

In [13]:
rawData[rawData.PRICE < 215.16]

,DATE,TIME_M,EX,SYM_ROOT,SYM_SUFFIX,TR_SCOND,SIZE,PRICE,TR_CORR,TR_SEQNUM,TR_ID,TR_SOURCE,TR_RF
0,20200108,4:00:00.024930588,P,QQQ,NaN,@ T,1803,214.78,0,1110,1,N,NaN
1,20200108,4:00:00.037031109,Q,QQQ,NaN,@FT,100,214.90,0,1111,1,N,NaN
2,20200108,4:00:00.037080526,P,QQQ,NaN,@FT,300,214.90,0,1112,2,N,NaN
3,20200108,4:00:00.037083089,P,QQQ,NaN,@FTI,50,215.02,0,1113,3,N,NaN
4,20200108,4:00:00.040577788,Q,QQQ,NaN,@FT,100,215.09,0,1117,2,N,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2048,20200108,8:01:09.999199535,D,QQQ,NaN,@ TI,6,214.88,0,6720,64,N,N
2049,20200108,8:01:10.023650368,D,QQQ,NaN,@ T,250,214.18,0,6732,65,N,N
2050,20200108,8:01:10.101645453,D,QQQ,NaN,@ TI,5,214.95,0,6782,66,N,N
2051,20200108,8:01:10.120803201,D,QQQ,NaN,@ TI,20,214.76,0,6800,67,N,N


In [14]:
taq.data.groupby(['ticker', 'date', 'exchange']).sum()

volume         price
ticker date       exchange                       
QQQ    2020-01-08 A           80262  1.760335e+05
                  B          293345  6.821704e+05
                  C          101533  1.832219e+05
                  D         7882878  5.593678e+06
                  J          307285  4.750841e+05
                  K         1986052  2.463048e+06
                  M         1535747  4.475115e+04
                  N          990829  9.343122e+05
                  P         3608724  4.191013e+06
                  Q         6560836  6.929210e+06
                  V          264999  4.107288e+05
                  X          417704  6.189801e+05
                  Y          648413  9.877919e+05
                  Z         2406811  3.242254e+06

## Ticker Specific

In [0]:
QQQ = data[data.SYM_ROOT == 'QQQ']

In [0]:
QQQ.shape

(35788, 13)

In [0]:
QQQ.EX.value_counts()

Q    10001
D     6265
P     6156
Z     4299
K     3659
Y     1193
N     1133
X      879
B      698
J      653
V      601
C      164
A       70
M       17
Name: EX, dtype: int64

In [0]:
IBM = data[data.SYM_ROOT == 'IBM']

In [0]:
IBM.EX.value_counts()

D    2129
Z    1283
N    1101
T    1032
K     686
P     582
Y     395
J     327
V     116
B      38
C      18
X      11
A       2
M       1
Name: EX, dtype: int64

In [0]:
IBM[IBM.DATE == 20200103]

,DATE,TIME_M,EX,SYM_ROOT,SYM_SUFFIX,TR_SCOND,SIZE,PRICE,TR_CORR,TR_SEQNUM,TR_ID,TR_SOURCE,TR_RF
